Download SEC filing data

In [1]:
from sec_edgar_downloader import Downloader
import os
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# Initialize a downloader instance. If no argument is passed
# to the constructor, the package will download filings to
# the current working directory.

# dl = Downloader("/path/to/valid/save/location")

currentDirectory = os.getcwd()
print(currentDirectory)
dl = Downloader(currentDirectory + '/data/download/')

Download dow30 10-K 10-Q

In [ ]:
dow30 = ['AAPL','AMGN','AXP','BA','CAT','CRM','CSCO','CVX','DIS','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','KO','MCD','MMM','MRK','MSFT','NKE','PG','TRV','UNH','V','VZ','WBA','WMT']
for x in dow30:
    dl.get("10-Q",x,amount=6)
    dl.get("10-K",x,amount=5)

Functions to parse report

In [48]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)



In [80]:
def parse_10k(filepath): 
    f = open(filepath)
    raw_10k = f.read()
    
    ## find start end range to avoid xml parsing problems ##
    import re
    # Regex to find <DOCUMENT> tags
    doc_start_pattern = re.compile(r'<DOCUMENT>')
    doc_end_pattern = re.compile(r'</DOCUMENT>')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    type_pattern = re.compile(r'<TYPE>[^\n]+')
    doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
    doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]
    
    doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
    
    document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
        if doc_type == '10-K':
            #print(doc_start)
            #print(doc_end)
            doc_10k = raw_10k[doc_start:doc_end].lower()
            
    doc_10k_bs = BeautifulSoup(doc_10k, 'lxml')
    doc_10k_str = str(doc_10k_bs)
    
    i7 = "management’s discussion and analysis of financial condition and results of operations"
    sec7_find = re.finditer(i7, doc_10k_str)
    i7a = 'quantitative and qualitative disclosures about market risk' 
    sec7a_find = re.finditer(i7a, doc_10k_str)

    # moving keyword range to dataframe
    sec7_start_range = []
    sec7_end_range = []
    for x in sec7_find: 
        #print(x.start())
        sec7_start_range.append(x.start())
        sec7_end_range.append(x.end())
        sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
    sec7a_start_range = []
    sec7a_end_range = []    
    for x in sec7a_find: 
        sec7a_start_range.append(x.start())
        sec7a_end_range.append(x.end())
        sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})

    #qc check
    #for index, row in sec7_df.iterrows():
    #    print(row['start'], row['end'])
    #    print('7aaaa')
        #for index, row in sec7a_df.iterrows():
        #    print(row['start'], row['end'])
        
    # get possible ranges 
    possible_start_range = []
    possible_end_range = []
    for index, row7a in sec7a_df.iterrows():
        filtered = []
        for i, row7 in sec7_df.iterrows():
            if row7['end'] < row7a['start']:
                filtered.append(row7['end'])
        #print(filtered)
        max_y = max(filtered)
        possible_start_range.append(max_y)
        possible_end_range.append(row7a['start']-1)

        # qc 
        #for x in possible_start_range:
        #    print(x)

        # get the largest possible range
    possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
    possible_df['delta'] = possible_df['end'] - possible_df['start']

    item7_df = possible_df.loc[possible_df['delta'].idxmax()]
    
    item_7_content = doc_10k[item7_df['start']:item7_df['end']]
    item_7_cleaned = preprocess(item_7_content)
    bagOfWordsA = item_7_cleaned.split(' ')
    
    return bagOfWordsA


In [83]:
def read_file_date(filepath):
    # Regex to find <DOCUMENT> tags
    date_start_pattern = re.compile(r'FILED AS OF DATE:')
    date_end_pattern = re.compile(r'DATE AS OF CHANGE:')
    # Regex to find <TYPE> tag prceeding any characters, terminating at new line
    date_type_pattern = re.compile(r'<TYPE>[^\n]+')

    f = open(filepath, "r")

    raw_10k = f.read()
    document = {}

    date_start_is = [x.end() for x in date_start_pattern.finditer(raw_10k)]
    date_end_is = [x.start() for x in date_end_pattern.finditer(raw_10k)]
    date_types = [x[len('<TYPE>'):] for x in date_type_pattern.findall(raw_10k)]

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
    for date_type, date_start, date_end in zip(date_types, date_start_is, date_end_is):
        if date_type == '10-K':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
        if date_type == '10-Q':
            ddd = raw_10k[date_start:date_end]
            ddd = ddd.strip()
    return ddd

Parse 10K 

In [85]:
dow30 = ['AAPL','AMGN','AXP','BA','CAT','CRM','CSCO','CVX','DIS','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','KO','MCD','MMM','MRK','MSFT','NKE','PG','TRV','UNH','V','VZ','WBA','WMT']
#dow30 = ['AAPL','AMGN','AXP']
companyk = []
file_namek = []
doc_typek = []
file_datek = []
contentk = []
for x in dow30: # company loop
    i = 0
    currentDirectory = os.getcwd()
    file_dir = currentDirectory + '/data/download/'
    file_dir = file_dir + 'sec-edgar-filings/' + x + '/10-K/'
    for name in os.listdir(file_dir): 
        if not name.startswith('.') and os.path.isfile(os.path.join(file_dir,name)+'/full-submission.txt'):
            
            if i >=2: # only get 2 10K files from each company
                break
            print(os.path.join(file_dir,name)+'/full-submission.txt')
            item7 = parse_10k(os.path.join(file_dir,name)+'/full-submission.txt')
            print(len(item7))
            contentk.append(item7)
            companyk.append(x)
            file_namek.append(name)
            doc_typek.append('10-K')
            file_d = read_file_date(os.path.join(file_dir,name)+'/full-submission.txt')
            file_datek.append(file_d)
            i = i+1
            print(i)

/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AAPL/10-K/0000320193-19-000119/full-submission.txt
2143
1
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AAPL/10-K/0000320193-17-000070/full-submission.txt
4772
2
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AMGN/10-K/0000318154-20-000017/full-submission.txt
4369
1
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AMGN/10-K/0000318154-18-000004/full-submission.txt
5631
2
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AXP/10-K/0000004962-19-000018/full-submission.txt
12195
1
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/AXP/10-K/0000004962-20-000030/full-submission.txt
12220
2
/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/downloa

RecursionError: maximum recursion depth exceeded

step by step testing

In [100]:
filepath = '/Users/wailunchung/Documents/GitHub/DS6120_Capstone_A_dey_chung/data/download/sec-edgar-filings/CAT/10-K/0000018230-21-000063/full-submission.txt'


In [101]:
f = open(filepath)
raw_10k = f.read()

In [102]:
import re
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

In [103]:
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]

In [127]:
document = {}

    # Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K':
        print(doc_start)
        print(doc_end)
        #doc_10k = raw_10k[doc_start:doc_end].lower()
        document[doc_type] = raw_10k[doc_start:doc_end].lower()

1106
8444285
14209841
17674001


In [129]:
len(document['10-K'])

3464160

In [105]:
doc_10k = BeautifulSoup(doc_10k, 'lxml')

In [112]:
doc_10k = str(doc_10k)

In [111]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

5000


In [131]:
doc_10k = raw_10k[1106:8444285]

In [142]:
i7 = "discussion and analysis of financial condition and results of operations"
i7 = "financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k)

In [143]:
for x in sec7_find: 
    print(x.start())

1087998
1132112


In [145]:
doc_10k[1087900:1132112]

'uding successfully integrating acquired businesses, could have an adverse effect on our business, financial condition and results of operations.&#160; Furthermore, we make strategic divestitures from time to time. In the case of divestitures, we may agree to indemnify acquiring parties for certain liabilities arising from our former businesses. These divestitures may also result in continued financial involvement in the divested businesses following the transaction, including through guarantees or other financial arrangements.&#160; Lower performance by those divested businesses could affect our future financial results.</span></div><div><span><br/></span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font-weight:700;line-height:120%">Union disputes or other labor matters could adversely affect our operations and financial results.</span></div><div><span style="color:#000000;font-family:\'Times New Roman\',sans-serif;font-size:10pt;font

In [133]:
#i7 = 'management&#8217;s discussion and analysis of financial condition and results of operations'
i7 = "management’s discussion and analysis of financial condition and results of operations"
sec7_find = re.finditer(i7, doc_10k)
i7a = 'quantitative and qualitative disclosures about market risk' 
sec7a_find = re.finditer(i7a, doc_10k)
    
# moving keyword range to dataframe
sec7_start_range = []
sec7_end_range = []
for x in sec7_find: 
    print(x.start())
    print(x.end())
    sec7_start_range.append(x.start())
    sec7_end_range.append(x.end())
    sec7_df = pd.DataFrame({'start': sec7_start_range,'end': sec7_end_range})
sec7a_start_range = []
sec7a_end_range = []    
for x in sec7a_find: 
    print(x.start())
    print(x.end())
    sec7a_start_range.append(x.start())
    sec7a_end_range.append(x.end())
    sec7a_df = pd.DataFrame({'start': sec7a_start_range,'end': sec7a_end_range})
    
#qc check
#for index, row in sec7_df.iterrows():
#    print(row['start'], row['end'])
#    print('7aaaa')
    #for index, row in sec7a_df.iterrows():
    #    print(row['start'], row['end'])


In [114]:
    
# get possible ranges 
possible_start_range = []
possible_end_range = []
for index, row7a in sec7a_df.iterrows():
    filtered = []
    for i, row7 in sec7_df.iterrows():
        if row7['end'] < row7a['start']:
            filtered.append(row7['end'])
    #print(filtered)
    max_y = max(filtered)
    possible_start_range.append(max_y)
    possible_end_range.append(row7a['start']-1)
    
    # qc 
    #for x in possible_start_range:
    #    print(x)
    
    # get the largest possible range
possible_df = pd.DataFrame({'start': possible_start_range,'end': possible_end_range})
possible_df['delta'] = possible_df['end'] - possible_df['start']
    
possible_df = possible_df.loc[possible_df['delta'].idxmax()]


In [115]:
possible_df

start    45884
end      46460
delta      576
Name: 0, dtype: int64

In [116]:
item_7_content = doc_10k[possible_df['start']:possible_df['end']]
sss = preprocess(item_7_content)
bagOfWordsA = sss.split(' ')

In [117]:
bagOfWordsA

['amp',
 'vgp',
 'amp',
 'jmo',
 'xouqw',
 'cqgi',
 'amp',
 'hwb',
 'amp',
 'ebedj',
 'hly',
 'mfdh',
 'amp',
 'ipftsh',
 'yck',
 'ynn',
 'yphbtxx',
 'tksj',
 'lgf',
 'gwonu',
 'amp',
 'mwekkacmz',
 'amp',
 'naibt',
 'kkf',
 'bizxh',
 'amp']